# **Inference RT-DETR**

In [1]:
import torch

# CUDAが使えるかどうかを確認
cuda_available = torch.cuda.is_available()

if cuda_available:
    print(f"CUDA is available! GPU: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available.")
     

CUDA is available! GPU: Quadro RTX 5000


In [2]:
#環境の確認
import torch
import sys
print(f"Python version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"PyTorch CUDA version: {torch.version.cuda}")

Python version: 3.8.6 (tags/v3.8.6:db45529, Sep 23 2020, 15:52:53) [MSC v.1927 64 bit (AMD64)]
PyTorch version: 2.4.1+cu121
CUDA available: True
PyTorch CUDA version: 12.1


In [ ]:
# Inference London-bus image
from ultralytics import RTDETR
import wget
import cv2
import matplotlib.pyplot as plt
import os

# RTDETRモデルのダウンロードと初期化
model_path = r'models\rtdetr-l.pt'

try:
    model = RTDETR(model_path)
except:
    # モデルディレクトリが存在しない場合は作成
    os.makedirs(os.path.dirname(model_path), exist_ok=True)
    
    # モデルをダウンロードして指定したパスに保存
    wget.download('https://github.com/ultralytics/assets/releases/download/v0.0.0/rtdetr-l.pt', model_path)
    model = RTDETR(model_path)

# 推論実行
image_path = 'london-buses.jpg'
results = model(image_path)

# 結果の表示と保存
for r in results:
    # 画像をプロットして表示
    # plt.figure(figsize=(12, 8))
    # plt.imshow(cv2.cvtColor(r.plot(), cv2.COLOR_BGR2RGB))
    # plt.axis('off')
    # #plt.show()
    
    # 結果の保存
    cv2.imwrite('result.jpg', r.plot())


image 1/1 c:\Users\ykita\ROP_AI_project\london-buses.jpg: 640x640 16 persons, 2 cars, 1 bus, 249.8ms
Speed: 7.0ms preprocess, 249.8ms inference, 13.7ms postprocess per image at shape (1, 3, 640, 640)


# **Test new model**

In [ ]:
from ultralytics import RTDETR
import cv2
import matplotlib.pyplot as plt

model = RTDETR(r"C:\Users\ykita\ROP_AI_project\models\rtdetr-l-1699_1703.pt")

# 推論実行
image_path = r'C:\Users\ykita\ROP_AI_project\ROP_image\IMG_1698\IMG_1698_0003.jpg'
results = model(image_path, save=False)

for r in results:
    boxes = r.boxes  # YOLOv8の場合

    # YOLO形式出力
    for box in boxes:
        cls_id = int(box.cls[0])  # クラスID
        x_center, y_center, width, height = box.xywhn[0]

        # YOLO形式: class x_center y_center width height
        # 小数点以下6桁で出力したい場合
        print(f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    #検出結果を画像に描画して表示
    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(r.plot(), cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

    #結果の保存
    cv2.imwrite('result.jpg', r.plot())


In [ ]:
#Web画像のinference

from ultralytics import RTDETR
import wget
import cv2
import matplotlib.pyplot as plt
import requests
from PIL import Image
from io import BytesIO

# モデルが未定義ならロードする
try:
    model  # 変数modelが定義されているか確認
except NameError:
    print("modelが未定義のため、ロードを実行します。")
    model = RTDETR(r"C:\Users\ykita\FacePhoto_instance\models\135best_rtdetr.pt")

# 画像URLの指定
url = "https://365dentist.jp/wp-content/uploads/2023/06/26579868_s.jpg"

# 画像のダウンロード
response = requests.get(url)
img = Image.open(BytesIO(response.content))

# 推論実行
results = model(img)

# 結果の表示と保存
for r in results:
    # 画像をプロットして表示
    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(r.plot(), cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

# **RT-DETR トレーニングフォルダ構成**
```
準備する形
project_root/
│
└── data/
    |
    │
    ├── images/ --> データセットから移行
    │   ├── img1.jpg
    │   ├── img2.jpg
    │   └── ...
    │   
    ├── labels/ --> CVATからそのまま移す      　　　
    |       ├── img1.txt
    |       ├── img2.txt
    |       └── ...
    |
    | 
    ├── Train.txt　　　　　　# トレーニングファイルのリスト --> CVATからそのまま移す
    │
    └── dataset.yaml        # データセット設定ファイル  --> 新たに作成




最終形
project_root/
│
└── data/
    |
    │
    ├── train/
    │   ├── images/       # トレーニング用の画像
    │   │   ├── img1.jpg
    │   │   ├── img2.jpg
    │   │   └── ...
    │   │
    │   └── labels/      # トレーニング用のラベル（アノテーション）
    │       ├── img1.txt
    │       ├── img2.txt
    │       └── ...
    │
    ├── val/
    │   ├── images/      # 検証用の画像
    │   │   ├── img1.jpg
    │   │   └── ...
    │   │
    │   └── labels/     # 検証用のラベル
    │       ├── img1.txt
    │       └── ...
    │
    ├── test/           # (オプション) テスト用のデータセット
    |   ├── images/
    |   └── labels/
    │
    ├── Train.txt           # トレーニングファイルのリスト
    │
    └── dataset.yaml        # データセット設定ファイル:
```


## **dataset.yaml**

```
# train and val data as 1) directory: path/images/, 2) file: path/images.txt, or 3) list: [path1/images/, path2/images/]
train: ./data/images/train
val: ./data/images/valid

# number of classes
nc: 2

# class names:
  0: Right_eye
  1: Left_eye

```

In [ ]:
# このスクリプトは、指定されたディレクトリ内のラベルファイルと画像ファイルを
# 以下のようにコピーします：
#
# 1. source_labels_dir/LABELS_xxxx/ 内の全ファイル → destination_labels_dir/
# 2. source_images_dir/IMG_xxxx/ 内の全ファイル → destination_images_dir/
#
# 処理対象のディレクトリは、label_dirとimage_dirのリストで指定します。
# 各ディレクトリ内のファイルは全て、指定された出力先ディレクトリにコピーされます。
# tqdmを使用して、コピーの進捗状況をプログレスバーで表示します。

import shutil
import os
from tqdm import tqdm

# コピー元ディレクトリ
source_labels_dir = r'C:\Users\ykita\ROP_AI_project\ROP_image'
source_images_dir = r'C:\Users\ykita\ROP_AI_project\ROP_image'

# コピー先ディレクトリ
destination_labels_dir = r'C:\Users\ykita\ROP_AI_project\data\labels'
destination_images_dir = r'C:\Users\ykita\ROP_AI_project\data\images'

# 処理対象のディレクトリリスト
label_dir = ['LABELS_1703_lens_seg']
image_dir = ['IMG_1703_lens']


# destination_labels_dirとdestination_images_dirが存在する場合は削除し、新規作成する
if os.path.exists(destination_labels_dir):
    shutil.rmtree(destination_labels_dir)
os.makedirs(destination_labels_dir, exist_ok=True)

if os.path.exists(destination_images_dir):
    shutil.rmtree(destination_images_dir)
os.makedirs(destination_images_dir, exist_ok=True)

# ラベルファイルのコピー
for label_folder in tqdm(label_dir, desc="ラベルファイルのコピー中"):
    source_folder = os.path.join(source_labels_dir, label_folder)
    # フォルダ内の全ファイルを取得
    if os.path.isdir(source_folder):
        files = os.listdir(source_folder)
        for file in files:
            src = os.path.join(source_folder, file)
            dst = os.path.join(destination_labels_dir, file)
            shutil.copy(src, dst)
            print(f"コピーしました: {src} -> {dst}")

# 画像ファイルのコピー
for image_folder in tqdm(image_dir, desc="画像ファイルのコピー中"):
    source_folder = os.path.join(source_images_dir, image_folder)
    # フォルダ内の全ファイルを取得
    if os.path.isdir(source_folder):
        files = os.listdir(source_folder)
        for file in files:
            src = os.path.join(source_folder, file)
            dst = os.path.join(destination_images_dir, file)
            shutil.copy(src, dst)
            print(f"コピーしました: {src} -> {dst}")

In [1]:
#segmentationはcvatのラベルが1スタートなので、0スタートに修正
"""
# ラベルインデックス調整スクリプト
# 目的：YOLOフォーマットのアノテーションファイルのラベルインデックスを1始まりから0始まりに変更
# 対象：指定されたディレクトリ内のすべての.txtファイル
# 
# 入力ファイルの例：
# 1 0.399550 0.361382 0.482096 0.225870 ...  # ラベル1で始まる
# 2 0.452748 0.391491 0.488882 0.403540 ...  # ラベル2で始まる
#
# 出力ファイルの例：
# 0 0.399550 0.361382 0.482096 0.225870 ...  # ラベル0に変更
# 1 0.452748 0.391491 0.488882 0.403540 ...  # ラベル1に変更
#
# 使用方法：
# 1. ラベルファイルが含まれているディレクトリパスを設定
# 2. スクリプトを実行すると、すべての.txtファイルを処理
# 3. 元のファイルが直接更新されます
#
# 注意：元のファイルが直接更新されるため、実行前にファイルのバックアップを
# 取ることをお勧めします
"""

import os
from pathlib import Path

def adjust_labels(input_path):
    """
    ラベルファイルを読み込み、すべてのラベルインデックスを1減らします
    （1始まりから0始まりに変更）。
    元のファイルを調整後の内容で上書きします。
    
    引数：
        input_path (str): 入力ラベルファイルのパス
    """
    # 元のファイル内容を読み込み
    with open(input_path, 'r') as f:
        lines = f.readlines()
    
    adjusted_lines = []
    for line in lines:
        parts = line.strip().split()
        if parts:  # 空行をスキップ
            # ラベル（最初の数字）を1減らす
            adjusted_line = f"{int(parts[0]) - 1} {' '.join(parts[1:])}"
            adjusted_lines.append(adjusted_line)
    
    # 同じファイルに書き戻す
    with open(input_path, 'w') as f:
        f.write('\n'.join(adjusted_lines))

def process_directory(directory_path):
    """
    指定されたディレクトリ内のすべての.txtファイルを処理します。
    
    引数：
        directory_path (str): ラベルファイルを含むディレクトリのパス
    """
    directory = Path(directory_path)
    txt_files = list(directory.glob('*.txt'))
    total_files = len(txt_files)
    
    print(f"{directory_path} 内で {total_files} 個の.txtファイルが見つかりました")
    
    for i, file_path in enumerate(txt_files, 1):
        try:
            adjust_labels(str(file_path))
            print(f"処理完了 [{i}/{total_files}]: {file_path.name}")
        except Exception as e:
            print(f"エラーが発生しました {file_path.name}: {str(e)}")

# ディレクトリパス
labels_dir = r"C:\Users\ykita\ROP_AI_project\data\labels"

# ディレクトリ内のすべてのファイルを処理
process_directory(labels_dir)
print("\n処理が完了しました！")

C:\Users\ykita\ROP_AI_project\data\labels 内で 456 個の.txtファイルが見つかりました
処理完了 [1/456]: IMG_1703_0171.txt
処理完了 [2/456]: IMG_1703_0172.txt
処理完了 [3/456]: IMG_1703_0173.txt
処理完了 [4/456]: IMG_1703_0174.txt
処理完了 [5/456]: IMG_1703_0179.txt
処理完了 [6/456]: IMG_1703_0180.txt
処理完了 [7/456]: IMG_1703_0181.txt
処理完了 [8/456]: IMG_1703_0182.txt
処理完了 [9/456]: IMG_1703_0183.txt
処理完了 [10/456]: IMG_1703_0184.txt
処理完了 [11/456]: IMG_1703_0185.txt
処理完了 [12/456]: IMG_1703_0202.txt
処理完了 [13/456]: IMG_1703_0203.txt
処理完了 [14/456]: IMG_1703_0204.txt
処理完了 [15/456]: IMG_1703_0205.txt
処理完了 [16/456]: IMG_1703_0206.txt
処理完了 [17/456]: IMG_1703_0207.txt
処理完了 [18/456]: IMG_1703_0208.txt
処理完了 [19/456]: IMG_1703_0209.txt
処理完了 [20/456]: IMG_1703_0210.txt
処理完了 [21/456]: IMG_1703_0211.txt
処理完了 [22/456]: IMG_1703_0212.txt
処理完了 [23/456]: IMG_1703_0213.txt
処理完了 [24/456]: IMG_1703_0214.txt
処理完了 [25/456]: IMG_1703_0215.txt
処理完了 [26/456]: IMG_1703_0216.txt
処理完了 [27/456]: IMG_1703_0217.txt
処理完了 [28/456]: IMG_1703_0218.txt
処理完了 [29/456]: IM

In [2]:
# データの前処理
# 目的: 画像ファイルとラベルファイルの対応関係を確認
# 処理内容:
# 1. ラベルディレクトリとイメージディレクトリのパスを設定
# 2. 各ディレクトリからファイル名(拡張子なし)を取得
# 3. ラベルが存在しない画像ファイルを特定
# 4. 不一致の数を表示

import os

# ラベルとイメージのディレクトリパス
labels_dir = r"data\labels"
images_dir = r"data\images"

# ラベルファイルの basename (拡張子なし) を取得
label_files = {os.path.splitext(f)[0] for f in os.listdir(labels_dir) if f.endswith('.txt')}

# 画像ファイルの basename (拡張子なし) を取得
image_files = {os.path.splitext(f)[0] for f in os.listdir(images_dir) if f.endswith('.jpg')}

# ラベルが存在しない画像ファイルを見つける
images_without_labels = image_files - label_files

# 結果を表示
print(f"ラベルが存在しない画像ファイルの数: {len(images_without_labels)}")

ラベルが存在しない画像ファイルの数: 432


In [ ]:
# Detectionの場合は
# ラベルが存在しない画像ファイルを削除
for img_name in images_without_labels:
    img_path = os.path.join(images_dir, img_name + '.jpg')
    try:
        os.remove(img_path)
        print(f"削除しました: {img_path}")
    except OSError as e:
        print(f"削除に失敗しました {img_path}: {e}")

print("不一致ファイルの削除が完了しました")

In [ ]:
# Segmentationの場合は
# 画像が存在しないlabelファイルに空のtxtを割り当て
import os

# ディレクトリのパス
labels_dir = r'C:\Users\ykita\ROP_AI_project\data\labels'
images_dir = r'C:\Users\ykita\ROP_AI_project\data\images'

# ラベルファイルの basename (拡張子なし) を取得
label_files = {os.path.splitext(f)[0] for f in os.listdir(labels_dir) if f.endswith('.txt')}
print(f"label_files: {label_files}")
print(f"len(label_files): {len(label_files)}")

# 画像ファイルの basename (拡張子なし) を取得
image_files = {os.path.splitext(f)[0] for f in os.listdir(images_dir) if f.endswith('.jpg')}
print(f"image_files: {image_files}")
print(f"len(image_files): {len(image_files)}")

# ラベルが存在しない画像ファイルを見つける
labels_without_labels = image_files - label_files
print(labels_without_labels)

# 空のテキストファイルを割り当て
for label in labels_without_labels:
    label_path = os.path.join(labels_dir, label + '.txt')
    with open(label_path, 'w') as f:
        pass  # 空のファイルを作成
    print(f"空のテキストファイルを作成しました: {label_path}")

In [4]:
# 1. 指定されたディレクトリから画像ファイル(.jpg, .png, .tif)を取得
# 2. 画像ファイルをランダムに並び替え
# 3. train/validディレクトリを作成
# 4. データを8:2の比率で分割
# 5. 画像ファイルと対応するラベルファイル(.txt)を適切なディレクトリに移動

import os
import shutil
import random
from tqdm import tqdm

# データディレクトリのパス設定
data_img_dir = r"data\images"
data_label_dir = r"data\labels"
src_img_dir = r"data\images"
src_label_dir = r"data\labels"

# ソースディレクトリの画像ファイルを直接取得（サブディレクトリを除く）
image_files = [f for f in os.listdir(src_img_dir) 
               if f.lower().endswith(('.jpg', '.jpeg', '.png', '.tif', '.tiff')) 
               and os.path.isfile(os.path.join(src_img_dir, f))]

if not image_files:
    print("エラー: 画像ファイルが見つかりません")
else:
    print(f"見つかった画像ファイル数: {len(image_files)}")
    
    # train/validディレクトリの作成
    for split in ['train', 'valid']:
        img_split_dir = os.path.join(data_img_dir, split)
        label_split_dir = os.path.join(data_label_dir, split)
        
        os.makedirs(img_split_dir, exist_ok=True)
        os.makedirs(label_split_dir, exist_ok=True)

    # データの分割
    random.shuffle(image_files)
    train_size = int(len(image_files) * 0.8)
    train_files = image_files[:train_size]
    valid_files = image_files[train_size:]

    # ファイルの移動関数
    def move_files(files, split):
        for img_file in tqdm(files, desc=f"{split}データの移動"):
            try:
                # 画像ファイルの移動
                src_img_path = os.path.join(src_img_dir, img_file)
                dst_img_path = os.path.join(data_img_dir, split, img_file)
                
                if os.path.exists(src_img_path):
                    shutil.move(src_img_path, dst_img_path)

                # 対応するラベルファイルの移動
                label_file = os.path.splitext(img_file)[0] + '.txt'
                src_label_path = os.path.join(src_label_dir, label_file)
                dst_label_path = os.path.join(data_label_dir, split, label_file)
                
                if os.path.exists(src_label_path):
                    shutil.move(src_label_path, dst_label_path)

            except Exception as e:
                print(f"エラーが発生しました ({img_file}): {str(e)}")

    # trainとvalidそれぞれにファイルを移動
    move_files(train_files, 'train')
    move_files(valid_files, 'valid')

    print(f"訓練データ数: {len(train_files)}")
    print(f"検証データ数: {len(valid_files)}")
    print("データの分割と移動が完了しました")

見つかった画像ファイル数: 888


validデータの移動: 100%|██████████| 178/178 [00:00<00:00, 1147.97it/s]

訓練データ数: 710
検証データ数: 178
データの分割と移動が完了しました


In [5]:
import os

# ディレクトリのパス
train_images_dir = r'C:\Users\ykita\ROP_AI_project\data\images\train'
train_labels_dir = r'C:\Users\ykita\ROP_AI_project\data\labels\train'
val_images_dir = r'C:\Users\ykita\ROP_AI_project\data\images\valid'
val_labels_dir = r'C:\Users\ykita\ROP_AI_project\data\labels\valid'

# 対応の確認と削除
def check_and_remove_unmatched_files(images_dir, labels_dir):
    image_files = set(os.path.splitext(f)[0] for f in os.listdir(images_dir) if f.endswith('.jpg'))
    label_files = set(os.path.splitext(f)[0] for f in os.listdir(labels_dir) if f.endswith('.txt'))

    unmatched_images = image_files - label_files
    unmatched_labels = label_files - image_files

    # 対応のない画像ファイルを削除
    for image in unmatched_images:
        image_path = os.path.join(images_dir, image + '.jpg')
        if os.path.exists(image_path):
            print(f"画像ファイル {image_path} に対応するラベルがありません。削除しますか？ (y/n): ", end="")
            if input().strip().lower() == 'y':
                os.remove(image_path)
                print(f"{image_path} を削除しました。")

    # 対応のないラベルファイルを削除
    for label in unmatched_labels:
        label_path = os.path.join(labels_dir, label + '.txt')
        if os.path.exists(label_path):
            print(f"ラベルファイル {label_path} に対応する画像がありません。削除しますか？ (y/n): ", end="")
            if input().strip().lower() == 'y':
                os.remove(label_path)
                print(f"{label_path} を削除しました。")

# 訓練データと検証データの対応確認
check_and_remove_unmatched_files(train_images_dir, train_labels_dir)
check_and_remove_unmatched_files(val_images_dir, val_labels_dir)

### **YAMLファイルを作成 (for training)**

In [6]:
import yaml
import os

# data = {
#     'train': r'C:\Users\ykita\ROP_AI_project\data\images\train',
#     'val': r'C:\Users\ykita\ROP_AI_project\data\images\valid',
#     'nc': 1,
#     'names': {
#         0: 'Lens',
#     }
# }

data = {
    'train': r'C:\Users\ykita\ROP_AI_project\data\images\train',
    'val': r'C:\Users\ykita\ROP_AI_project\data\images\valid',
    'nc': 3,
    'names': {
        0: 'Fundus',
        1: 'Disc',
        2: 'Macula',
    }
}

yaml_path = 'data/data.yaml'

# dataディレクトリが存在しない場合は作成
os.makedirs('data', exist_ok=True)

# ファイルの存在確認
if os.path.exists(yaml_path):
    print(f"既存の {yaml_path} を上書きします。")
else:
    print(f"新規に {yaml_path} を作成します。")

# YAMLファイルの書き出し（既存ファイルは上書き）
with open(yaml_path, 'w', encoding='utf-8') as f:
    yaml.safe_dump(data, f, sort_keys=False, allow_unicode=True)

print("処理が完了しました。")

既存の data/data.yaml を上書きします。
処理が完了しました。


# **Train RT-DETR**

In [3]:
from ultralytics import RTDETR
from ultralytics import YOLO
import wget

# RTDETRモデルのダウンロードと初期化
try:
    #model = RTDETR('rtdetr-l.pt')
    # model = YOLO("yolo11lpt")  # load a pretrained model (recommended for training)
    model = YOLO("yolo11m-seg.pt")  # load a pretrained model (recommended for training)
except:
    #wget.download('https://github.com/ultralytics/assets/releases/download/v0.0.0/rtdetr-l.pt')
    #model = RTDETR('rtdetr-l.pt')
    # wget.download('https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11l.pt')
    # model = YOLO("yolo11l.pt")  # load a pretrained model (recommended for training)
    wget.download('https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11m-seg.pt')
    model = YOLO(r"C:\Users\ykita\ROP_AI_project\runs\segment\train9\weights\last.pt")  # load a pretrained model (recommended for training)
    
# Fine-tune
results = model.train(data=r'data\data.yaml', epochs=1000, patience=100, batch=8, imgsz=640, device=0, degrees=180, flipud=0.5, fliplr=0.5)

New https://pypi.org/project/ultralytics/8.3.70 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.59  Python-3.8.6 torch-2.4.1+cu121 CUDA:0 (Quadro RTX 5000, 16384MiB)
engine\trainer: task=segment, mode=train, model=yolo11m-seg.pt, data=data\data.yaml, epochs=1000, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train10, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=Fals

train: Scanning C:\Users\ykita\ROP_AI_project\data\labels\train.cache... 710 images, 351 backgrounds, 0 corrupt: 100%|██████████| 710/710 [00:00<?, ?it/s]
val: Scanning C:\Users\ykita\ROP_AI_project\data\labels\valid.cache... 178 images, 81 backgrounds, 0 corrupt: 100%|██████████| 178/178 [00:00<?, ?it/s]


Plotting labels to runs\segment\train10\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 115 weight(decay=0.0), 126 weight(decay=0.0005), 125 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\segment\train10
Starting training for 1000 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


     1/1000      5.23G      1.378      3.313      3.014      1.741         22        640: 100%|██████████| 89/89 [00:36<00:00,  2.41it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:06<00:00,  1.72it/s]


                   all        178        219      0.666      0.347      0.342      0.267      0.679      0.352      0.347      0.241

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


...

   504/1000      5.28G     0.6425      1.002     0.3372      1.036         13        640: 100%|██████████| 89/89 [00:30<00:00,  2.88it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:03<00:00,  3.91it/s]

                   all        178        219      0.967      0.915      0.948      0.682       0.96      0.909      0.941       0.67
EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 404, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



504 epochs completed in 4.759 hours.
Optimizer stripped from runs\segment\train10\weights\last.pt, 45.2MB
Optimizer stripped from runs\segment\train10\weights\best.pt, 45.2MB

Validating runs\segment\train10\weights\best.pt...
Ultralytics 8.3.59  Python-3.8.6 torch-2.4.1+cu121 CUDA:0 (Quadro RTX 5000, 16384MiB)
YOLO11m-seg summary (fused): 330 layers, 22,337,625 parameters, 0 gradients, 123.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:04<00:00,  2.42it/s]


                   all        178        219      0.951      0.897      0.946      0.689      0.951      0.897      0.943      0.682
                Fundus         97         97       0.98      0.979      0.994      0.921       0.98      0.979      0.994      0.902
                  Disc         68         68      0.994      0.897      0.979      0.726      0.994      0.897      0.978      0.701
                Macula         54         54      0.877      0.815      0.865      0.418      0.877      0.815      0.857      0.444
Speed: 0.6ms preprocess, 17.3ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to runs\segment\train10
